In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''

1. 텍스트 전처리

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# 단어 집합 생성 
t = Tokenizer()
t.fit_on_texts([text])

In [ ]:
## 특징 
# 1) 가나다 순서 X --> 많이 나온 단어 순서
# 2) 1 부터 시작함 
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [ ]:
# 단어 집합 크기 - Keras Toeknizer의 시작 인덱스가 1이기 때문에 1을 더해주어야 함 
vocab_size = len(t.word_index) + 1
vocab_size

12

In [ ]:
t.texts_to_sequences(['그의 말이 법이다.'])
# '그의': 6 / '말이': 1 / '법이다': 7

[[6, 1, 7]]

In [ ]:
# corpus --> encoding 
# 라인별로 끊는 경우
for line in text.split('\n'):
  encoded = t.texts_to_sequences([line])[0] # line -> 리스트로 감싸줘야 해!
  print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


In [ ]:
# 전체(text)를 적용하면?
t.texts_to_sequences([text])

[[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]]

In [ ]:
# [2, 3, 1, 4, 5]  : 경마장에 있는 말이 뛰고 있다.

#                       [   X      , y]
# [2, 3]             => [0, 0, 0, 2, 3]  '경마장에': 2 --> '있는': 3
# [2, 3, 1]             [0, 0, 2, 3, 1]  '경마장에': 2, '있는': 3 --> '말이': 1
# [2, 3, 1, 4]          [0, 2, 3, 1, 4]  '경마장에': 2, '있는': 3, '말이': 1 --> '뛰고': 4
# [2, 3, 1, 4, 5]       [2, 3, 1, 4, 5]  '경마장에': 2, '있는': 3, '말이': 1 , '뛰고': 4 --> '있다': 5

In [ ]:
# 라인별 sequences 생성 
sequences = []
for line in text.split('\n'):
  encoded = t.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequences.append(encoded[:i+1])
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [ ]:
# sequence 중 제일 길이가 긴 것 
max_len = max(len(seq) for seq in sequences)
max_len

6

2. Padding
- 전체 샘플 길이를 `max_len`에 맞추도록 0을 추가
- 앞 부분에 채우는 `pre` option -> default

In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

3. X, y data 설정 

In [ ]:
X = sequences[:, :-1]
y = sequences[:, -1]

X.shape, y.shape

((11, 5), (11,))

In [ ]:
Y = to_categorical(y)
Y.shape

(11, 12)

### Model
- `Embedding` -> input shape 걱정 안해도 됨
- `SimpleRNN`
- `Dense`

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [ ]:
# Embedding = 4, SimpleRNN = 32
model = Sequential([
                    Embedding(vocab_size, 4, input_length=max_len-1),
                    SimpleRNN(32),                                    # activation='tanh' -> default
                    Dense(vocab_size, activation='softmax')           # 다중 분류라서 activation='softmax'
                    ]) 
model.summary()

# Embedding() --> input=vocab_size, output_dim=4: Embedding vector 갯수(RNN입장에서 피처 개수),  input_lenght=nam_len-1: 맨 마지막을 y로 빼서!
# Embedding vector 갯수 = RNN layer입장에서 feature 개수

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 4)              48        
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1184      
                                                                 
 dense_1 (Dense)             (None, 12)                396       
                                                                 
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [ ]:
hist = model.fit(X, Y, epochs=200, verbose=0)

In [ ]:
hist.history['accuracy'][-1]    # accuracy: 100%

1.0

- Validaiton

In [ ]:
from google.colab import files
up = files.upload()
# my_util.py 파일 업로드 하기!

In [ ]:
from my_util import sentence_generation
# model, Tokenizer, maxlen, current text, loop num
sentence_generation(model, t, max_len, '경마장에', 3) 

'경마장에 있는 말이 뛰고'

In [ ]:
sentence_generation(model, t, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

#### Variation of Model 
- Embedding vector 갯수: [2, 4, 6]
- RNN 유닛 갯수: [24, 32, 48]

In [ ]:
for n_embed in [2, 4, 6]:
  for n_unit in [24, 32, 48]:
    model = Sequential([
                    Embedding(vocab_size, n_embed, input_length=max_len-1), # input=vocab_size, output_dim=4: 차원 축소,  input_lenght=nam_len-1: 맨 마지막을 y로 빼서!
                    SimpleRNN(n_unit),                                    # activation='tanh' -> default
                    Dense(vocab_size, activation='softmax')           # 다중 분류라서 activation='softmax'
                    ]) 
    model.compile('adam', 'categorical_crossentropy', ['accuracy'])
    hist = model.fit(X, Y, epochs=200, verbose=0)

    print('='*100)
    print(f'Embedding vector: {n_embed}, RNN unit: {n_unit}')
    print(f"정확도: {hist.history['accuracy'][-1]:.4f}")
    print(sentence_generation(model, t, max_len, '경마장에', 3))
    print(sentence_generation(model, t, max_len, '그의', 2))
    print(sentence_generation(model, t, max_len, '가는', 5))

Embedding vector: 2, RNN unit: 24
정확도: 0.9091
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
Embedding vector: 2, RNN unit: 32
정확도: 0.8182
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 고와야 오는 말이 곱다
Embedding vector: 2, RNN unit: 48
정확도: 0.9091
경마장에 말이 고와야 오는
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
Embedding vector: 4, RNN unit: 24
정확도: 0.9091
경마장에 있는 말이 뛰고
그의 말이 고와야
가는 말이 고와야 오는 말이 곱다
Embedding vector: 4, RNN unit: 32
정확도: 0.9091
경마장에 말이 고와야 오는
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
Embedding vector: 4, RNN unit: 48
정확도: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
Embedding vector: 6, RNN unit: 24
정확도: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
Embedding vector: 6, RNN unit: 32
정확도: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
Embedding vector: 6, RNN unit: 48
정확도: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
